In [9]:
from langchain.agents import create_agent
from langchain_core.tools import tool
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langchain.agents.structured_output import ToolStrategy

from langgraph.checkpoint.memory import InMemorySaver 

from typing_extensions import TypedDict

from dotenv import load_dotenv
load_dotenv()

class EmailRequest(TypedDict):
    recipient: str
    subject: str
    body: str


In [62]:
@tool()
def send_email(recipient: str, subject: str, body: str) -> str:
    """Send an email to a recipient.
    Args:
        recipient: Email address of the recipient.
        subject: Subject line of the email.
        body: Body content of the email.
    Returns:
        Confirmation message.
    """
    return f"Email sent successfully to {recipient}"


agent = create_agent(
    model="openai:gpt-4o",
    tools=[send_email],
    system_prompt="You are a helpful assistant for Sydney that can send emails.",
    middleware=[HumanInTheLoopMiddleware(interrupt_on={"send_email": True})],
    # response_format=ToolStrategy(EmailRequest),
    checkpointer=InMemorySaver(), # In production, use a persistent checkpointer like AsyncPostgresSaver.
)

In [57]:
# Choose decision
# Approval decision
approval = {
    "decisions": [
        {
            "type": "approve"
        }
    ]
}

# Edit decision
edit = {
    "decisions": [
        {
            "type": "edit",
            "edited_action": {
                "name": "send_email",
                "args": {
                    "recipient": "partner@startup.com",
                    "subject": "Budget proposal for Q1 2026",
                    "body": "I can only approve up to 500k, please send over details.",
                }
            }
        }
    ]
}

# Reject decision
reject = {
    "decisions": [
        {
            "type": "reject",
            "message": "Please edit the email asking for more details about the budget proposal, then send the email"
        }
    ]
}

decision = reject  # Change this to approval or reject to test other flows

In [68]:
import uuid
config = {"configurable": {"thread_id": str(uuid.uuid4())}}


In [72]:

# user_input = "Send email to alice@example.com with subject Meeting and body Let's meet at 10 AM tomorrow."
# user_input += " The budget proposal for Q1 2026 is 1 million dollars."

user_input = "add more to the body and include a joke"
result = agent.invoke(
    {"messages": [{"role": "user", "content": user_input}]},
    config=config
)

'''for msg in result["messages"]:
    msg.pretty_print()'''
'''
from langgraph.types import Command
if "__interrupt__" in result:
    description = result['__interrupt__'][-1].value['action_requests'][-1]['description']
    print("Human intervention required:", description)

    result = agent.invoke(
        Command(
            resume=decision
        ),
        config=config,    
    )'''

for msg in result["messages"]:
    msg.pretty_print()


================================ Human Message =================================

hi can you help me send an email to
================================== Ai Message ==================================

Sure, I can help you send an email. Could you please provide me with the recipient's email address, the subject of the email, and the body content you'd like to include?
================================ Human Message =================================

send an email to leo@gmail.com
================================== Ai Message ==================================

Please provide the subject and body content of the email you'd like to send to leo@gmail.com.
================================ Human Message =================================

email him about a meeting next week regarding the budget proposal for Q1 2026.
================================== Ai Message ==================================

Here's a draft of the email:

- **Recipient**: leo@gmail.com
- **Subject**: Meeting Next Week Rega